In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'



In [2]:
--- FUNCTIONS
--- Comparing with 2000
function compare_cfrs(tree_cfr1,tree_cfr2000)
    local difference = {}
    local table1 = tree_cfr1.total_strategies
    local table2 = tree_cfr2000.total_strategies
    
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[#table2]),2)))
    end
    return torch.FloatTensor(difference)
end
--- Comparing 2000 with itself
function get_convergence(tree_cfr)
    local difference = {}
    local table_cfr = tree_cfr.total_strategies
    local end_strategy = table_cfr[#table_cfr]
    
    for i = 1, #table_cfr do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table_cfr[i] - end_strategy),2)))
    end
    return torch.FloatTensor(difference)
end

In [3]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

### 1. Exploitability for original CFR and modified CFR

In [4]:
--- Building CFR Solver
tree2000 = builder:build_tree(params)
tree2000_cfr = TreeCFR()
tree2000_cfr:run_cfr(tree2000, starting_ranges,2000)

In [5]:
--- Building CFR Solver
tree2001 = builder:build_tree(params)
tree2001_cfr = TreeCFR2()
tree2001_cfr:run_cfr(tree2001, starting_ranges,2000)

In [6]:
exploit_2000_original = tree2000_cfr.exploitability_vec
exploit_2000_mod = tree2001_cfr.exploitability_vec

In [7]:
local num_iter = torch.range(1,exploit_2000_original:size(1))
local plot = Plot()

plot:line(num_iter,exploit_2000_original,'red' ,'Original CFR')
plot:line(num_iter,exploit_2000_mod,'blue' ,'Modified CFR')
plot:legend(true):title('Exploitability of the CFRs - 2000 iterations')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()


### 2. Exploitability for different Strategies

In [8]:
number_of_iterations = 10

In [9]:
--- Get Data from 2000 iterations
tree800 = builder:build_tree(params)
tree_data800 = TreeData(tree800)
tree800_cfr = TreeCFR()
tree800_cfr:run_cfr(tree800, starting_ranges,number_of_iterations)
tree_data800:get_training_set(tree800,1)

### Training the Model

In [10]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [11]:
current_criterion = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,10000)

In [12]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

### No Warm Start

In [13]:
--- Building CFR Solver
treeNoWarm = builder:build_tree(params)
treeNoWarm_cfr = TreeCFR2()
print("Solver")
treeNoWarm_cfr:run_cfr(treeNoWarm, starting_ranges,number_of_iterations)

0.011948538741145


Solver


### Neural Net Start 

In [14]:
--- NNWarm Starting
treeNNWarm = builder:build_tree(params)
dataNN = TreeData(treeNNWarm)
dataNN:get_training_set(treeNNWarm,1)
dataNN:generate_new_initial_strategies(treeNNWarm,nn_trainer)

In [15]:
--- Building CFR Solver
treeNNWarm_cfr = TreeCFR2()
treeNNWarm_cfr:run_cfr(treeNNWarm, starting_ranges,number_of_iterations)

### Target Warm Start 

In [16]:
--- Target Warm Starting
treeTargWarm = builder:build_tree(params)
dataTarg = TreeData(treeTargWarm)
dataTarg:get_training_set(treeTargWarm,1)
dataTarg:warm_start_targets(treeTargWarm,tree800)

In [17]:
--- Building CFR Solver
treeTargWarm_cfr = TreeCFR2()
treeTargWarm_cfr:run_cfr(treeTargWarm, starting_ranges,number_of_iterations)

### Plot different starts

In [18]:
exploit_unif = tree800_cfr.exploitability_vec
exploit_NoW = treeNoWarm_cfr.exploitability_vec
exploit_NNW = treeNNWarm_cfr.exploitability_vec
exploit_TargW = treeTargWarm_cfr.exploitability_vec

In [19]:
local num_iter = torch.range(1,exploit_NoW:size(1))
local plot = Plot()

plot:line(num_iter,exploit_unif,'black' ,'Original CFR in the paper')
plot:line(num_iter,exploit_NoW,'red' ,'No Warm start CFR')
plot:line(num_iter,exploit_NNW,'blue' ,'Neural Net warm start CFR')
plot:line(num_iter,exploit_TargW,'green' ,'Warm start with target values')
plot:legend(true):title('Exploitability of different strategies, 800 iterations')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()


### Graphical Comparaison

### 3. Comparaision with 2000 CFR2

In [20]:
--- Get Data from 2000 iterations
tree802 = builder:build_tree(params)
tree_data802 = TreeData(tree802)
tree802_cfr = TreeCFR2()
tree802_cfr:run_cfr(tree802, starting_ranges,number_of_iterations)
tree_data802:get_training_set(tree802,1)

In [21]:
-- Building the neural net model
strategy_nn2 = StrategyNN()
nn_model2 = strategy_nn2.model
-- Building trainer:
nn_trainer2 = NNTrainer(tree_data802,nn_model2)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [22]:
current_criterion2 = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model2,current_criterion2,opt,10000)

In [23]:
--- No Warm Start
treeNoWarm2 = builder:build_tree(params)
treeNoWarm_cfr2 = TreeCFR2()
treeNoWarm_cfr2:run_cfr(treeNoWarm2, starting_ranges,number_of_iterations)

In [24]:
--- NNWarm Starting
treeNNWarm2 = builder:build_tree(params)
dataNN2 = TreeData(treeNNWarm2)
dataNN2:get_training_set(treeNNWarm2,1)
dataNN2:generate_new_initial_strategies(treeNNWarm2,nn_trainer2)

In [25]:
--- NNWarm CFR Solver
treeNNWarm_cfr2 = TreeCFR2()
treeNNWarm_cfr2:run_cfr(treeNNWarm2, starting_ranges,number_of_iterations)

In [26]:
--- Target Warm Starting
treeTargWarm2 = builder:build_tree(params)
dataTarg2 = TreeData(treeTargWarm2)
dataTarg2:get_training_set(treeTargWarm2,1)
dataTarg2:warm_start_targets(treeTargWarm2,tree802)

In [27]:
--- Building CFR Solver
treeTargWarm_cfr2 = TreeCFR2()
treeTargWarm_cfr2:run_cfr(treeTargWarm2, starting_ranges,number_of_iterations)

In [28]:
exploit_unif2 = tree802_cfr.exploitability_vec
exploit_NoW2 = treeNoWarm_cfr2.exploitability_vec
exploit_NNW2 = treeNNWarm_cfr2.exploitability_vec
exploit_TargW2 = treeTargWarm_cfr2.exploitability_vec

In [29]:
local num_iter = torch.range(1,exploit_NoW:size(1))
local plot = Plot()

plot:line(num_iter,exploit_unif2,'black' ,'Original CFR in the paper')
plot:line(num_iter,exploit_NoW2,'red' ,'No Warm start CFR')
plot:line(num_iter,exploit_NNW2,'blue' ,'Neural Net warm start CFR')
plot:line(num_iter,exploit_TargW2,'green' ,'Warm start with target values')
plot:legend(true):title('Exploitability of different strategoes')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()
